# Hyperparameter optimization with Ray Tune

<a target="_blank" href="https://recognai.github.io/biome-text/v3.2.0/documentation/tutorials/3-Hyperparameter_optimization_with_Ray_Tune.html"><img class="icon" src="https://recognai.github.io/biome-text/v3.2.0/assets/img/biome-isotype.svg" width=24 /></a>
[View on recogn.ai](https://recognai.github.io/biome-text/v3.2.0/documentation/tutorials/3-Hyperparameter_optimization_with_Ray_Tune.html)

<a target="_blank" href="https://colab.research.google.com/github/recognai/biome-text/blob/v3.2.0/docs/docs/documentation/tutorials/3-Hyperparameter_optimization_with_Ray_Tune.ipynb"><img class="icon" src="https://www.tensorflow.org/images/colab_logo_32px.png" width=24 /></a>
[Run in Google Colab](https://colab.research.google.com/github/recognai/biome-text/blob/v3.2.0/docs/docs/documentation/tutorials/3-Hyperparameter_optimization_with_Ray_Tune.ipynb)

<a target="_blank" href="https://github.com/recognai/biome-text/blob/v3.2.0/docs/docs/documentation/tutorials/3-Hyperparameter_optimization_with_Ray_Tune.ipynb"><img class="icon" src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png" width=24 /></a>
[View source on GitHub](https://github.com/recognai/biome-text/blob/v3.2.0/docs/docs/documentation/tutorials/3-Hyperparameter_optimization_with_Ray_Tune.ipynb)

When running this tutorial in Google Colab, make sure to install *biome.text* and *ray tune* first:

In [ ]:
!pip install -U pip
!pip install -U biome-text
exit(0)  # Force restart of the runtime

If you want to log your runs with [WandB](https://wandb.ai), don't forget to install its client and log in.

In [ ]:
!pip install wandb
!wandb login

::: tip Note

In this tutorial we will use a GPU by default.
So when running this tutorial in Google Colab, make sure that you request one (*Edit -> Notebook settings*).

:::

## Introduction

Here we will optimize the hyperparameters of the short-text classifier from [this tutorial](https://recognai.github.io/biome-text/v3.2.0/documentation/tutorials/1-Training_a_text_classifier.html), hence we recommend to have a look at it first before going through this tutorial.
For the Hyper-Parameter Optimization (HPO) we rely on the awesome [Ray Tune library](https://docs.ray.io/en/latest/tune.html#tune-index).

For a short introduction to HPO with Ray Tune you can have a look at this nice [talk](https://www.youtube.com/watch?v=VX7HvEoMrsA) by Richard Liaw. 
We will follow his terminology and use the term *trial* to refer to a training run of one set of hyperparameters. 

### Imports

First let's import all the stuff we need for this tutorial:

In [ ]:
import os
from biome.text import Pipeline, Dataset
from biome.text.configuration import TrainerConfiguration
from biome.text.hpo import TuneExperiment
from ray import tune


## Creating the datasets

As a first step we will download the training and validation data to our local machine, and create our datasets. 

In [ ]:
!curl -O https://biome-tutorials-data.s3-eu-west-1.amazonaws.com/text_classifier/business.cat.train.csv
!curl -O https://biome-tutorials-data.s3-eu-west-1.amazonaws.com/text_classifier/business.cat.valid.csv

In [ ]:
train_ds = Dataset.from_csv("business.cat.train.csv")
valid_ds = Dataset.from_csv("business.cat.valid.csv")

## Defining the pipeline and the search space

As mentioned in the introduction we will use the same pipeline configuration as used in the [base tutorial](https://recognai.github.io/biome-text/v3.2.0/documentation/tutorials/1-Training_a_text_classifier.html)).

To perform a random hyperparameter search (as well as a grid search) we simply have to replace the parameters we want to optimize with methods from the [Random Distributions API](https://docs.ray.io/en/latest/tune/api_docs/search_space.html#random-distributions-api) or the [Grid Search API](https://docs.ray.io/en/latest/tune/api_docs/search_space.html#grid-search-api) in our configuration dictionaries.
For a complete description of both APIs and how they interplay with each other, see the corresponding section in the [Ray Tune docs](https://docs.ray.io/en/latest/tune/api_docs/search_space.html). 

In our case we will tune 9 parameters:
- the output dimensions of our `word` and `char` features
- the dropout of our `char` feature
- the architecture of our pooler (*GRU* versus *LSTM*)
- number of layers and hidden size of our pooler, as well as if it should be bidirectional
- hidden dimension of our feed forward network
- and the learning rate

For most of the parameters we will provide discrete values from which Tune will sample randomly, while for the dropout and learning rate we will provide a continuous linear and logarithmic range, respectively.
Since we want to directly compare the outcome of the optimization with the base configuration of the [underlying tutorial](https://recognai.github.io/biome-text/v3.2.0/documentation/tutorials/1-Training_a_text_classifier.html), we will fix the number of epochs to 3.

Not all of the parameters above are worth tuning, but we want to stress the flexibility that *Ray Tune* and *biome.text* offers you.

::: tip Tip

Keep in mind that the learning rate "*is often the single most important hyper-parameter and one should always make sure that it has been tuned (up to approximately a factor of 2). ... If there is only time to optimize one hyper-parameter and one uses stochastic gradient descent, then this is the hyper-parameter that is worth tuning.*" ([Yoshua Bengio](https://arxiv.org/abs/1206.5533)).

:::

In the following configuration dictionaries we replaced the relevant parameters with tune's search spaces.

In [ ]:
# defining the search spaces in our pipeline config
pipeline_config = {
    "name": "german_business_names",
    "tokenizer": {
        "text_cleaning": {
            "rules": ["strip_spaces"]
        }
    },
    "features": {
        "word": {
            "embedding_dim": tune.choice([32, 64]),
            "lowercase_tokens": True,
        },
        "char": {
            "embedding_dim": 32,
            "lowercase_characters": True,
            "encoder": {
                "type": "gru",
                "num_layers": 1,
                "hidden_size": tune.choice([32, 64]),
                "bidirectional": True,
            },
            "dropout": tune.uniform(0, 0.5),
        },
    },
    "head": {
        "type": "TextClassification",
        "labels": list(set(train_ds["label"])),
        "pooler": {
            "type": tune.choice(["gru", "lstm"]),
            "num_layers": tune.choice([1, 2]),
            "hidden_size": tune.choice([32, 64]),
            "bidirectional": tune.choice([True, False]),
        },
        "feedforward": {
            "num_layers": 1,
            "hidden_dims": tune.choice([32, 64]),
            "activations": ["relu"],
            "dropout": [0.0],
        },
    },       
}

# defining the search spaces in our trainer config
trainer_config = TrainerConfiguration(
    optimizer={
        "type": "adam",
        "lr": tune.loguniform(0.001, 0.01)
    },
    max_epochs=3,
    monitor="validation_accuracy",
    monitor_mode="max"
)

We also want to make sure that the model with the highest accuracy is saved in the end, that is why we specified the `monitor` and `monitor_mode` argument in the trainer configuration.

::: tip Note

By default we will use a GPU if available.
If you do not want to use your GPU, just set `gpus=0` in your `TrainerConfiguration` above.

:::

## Starting the random search

Before starting our random hyperparameter search we first have to create a `TuneExperiment` instance with our configurations dicts and our datasets.
We also set a name that will mainly be used as project and experiment name for the integrated WandB and MLFlow logger, respectively.

Furthermore, we can pass on all the parameters available for the underlying [`tune.Experiment`](https://docs.ray.io/en/master/tune/api_docs/execution.html#tune-experiment) class:

- The number of trials our search will go through depends on the `num_samples` parameter.
In our case, a random search, it equals the number of trials, whereas in the case of a grid search the total number of trials is `num_samples` times the grid configurations (see the [Tune docs](https://docs.ray.io/en/latest/tune/api_docs/search_space.html#overview) for illustrative examples).

- The `local_dir` parameter defines the output directory of the HPO results and will also contain the training results of each trial (that is the model weights and metrics).

- The number of parallel running trials depends on your `resources_per_trial` configuration and your local resources.
The default value is `{"cpu": 1, "gpu": 0}` and results, for example, in 8 parallel running trials on a machine with 8 CPUs.
You can also use fractional values. To share a GPU between 2 trials, for example, pass on `{"gpu": 0.5}`. 

::: tip Note

Keep in mind: to run your HPO on GPUs, you have to specify them in the `resources_per_trial` parameter when calling `tune.run()`.
If you do not want to use a GPU, just set the value to 0 `{"cpu": 1, "gpu": 0}`.

:::



In [ ]:
my_random_search = TuneExperiment(
    pipeline_config=pipeline_config,
    trainer_config=trainer_config,
    train_dataset=train_ds,
    valid_dataset=valid_ds,
    name="My first random search",
    # `tune.Experiment` kwargs:
    num_samples=50,
    local_dir="tune_runs",
    resources_per_trial={"cpu": 1, "gpu": 0.5},
)

With our `TuneExperiment` object at hand, we simply have to pass it on to the [`tune.run`](https://docs.ray.io/en/master/tune/api_docs/execution.html#tune-run) function to start our random search.

In this tutorial we will perform a random search together with the [Asynchronous Successive Halving Algorithm (ASHA)](https://blog.ml.cmu.edu/2018/12/12/massively-parallel-hyperparameter-optimization/) to schedule our trials.
The Ray Tune developers advocate this `scheduler` as a good starting point for its aggressive termination of low-performing trials.
You can look up the available configurations in the [ASHAScheduler docs](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#asha-tune-schedulers-ashascheduler), here we will just use the default parameters.

We also have to specify on what `metric` to optimize and its `mode` (should the metric be *minimized* (`min`) or *maximized* (`max`) ). 
This should be the same as the `validation_metric` specified in your trainer configuration. This guarantees the alignment of the patience mechanism and the trial scheduler, and also makes sure that the best model weights correspond to the best metrics reported by ray tune.

The `progress_reporter` is a nice feature to keep track of the progress inside a Jupyter Notebook, for example.

::: tip Tip

You can reduce the size of the training output by specifying `progress_bar_refresh_rate=0` in your `TrainingConfiguration` to disable the progress bar.

:::

In [ ]:
analysis = tune.run(
    my_random_search,
    scheduler=tune.schedulers.ASHAScheduler(), 
    metric="validation_accuracy", 
    mode="max",
    progress_reporter=tune.JupyterNotebookReporter(overwrite=True)
)

::: tip Tip

You can also create an `Analysis` object from the output directory of the HPO run, once it has finished:
```python
from ray.tune.analysis.experiment_analysis import Analysis
analysis = Analysis(
    experiment_dir='tune_runs/My first random search")
    default_metric="validation_accuracy",
    default_mode="max",
)
```

:::

### Following the progress with tensorboard (optional)

Ray Tune automatically logs its results with [TensorBoard](https://www.tensorflow.org/tensorboard/).
We can take advantage of this and launch a TensorBoard instance before starting the hyperparameter search to follow its progress.
The `RayTuneTrainable` class will also log the metrics to MLFlow and WandB, if you prefer those platforms.

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir ./runs/tune

![Screenshot of TensorBoard with Ray Tune](./img/hpo_tensorboard.png)
*Screenshot of TensorBoard*

## Checking the results

The *analysis* object returned by `tune.run` can be accessed through a *pandas DataFrame*.

In [ ]:
analysis.dataframe()

Event though with 50 trials we visit just a small space of our possible configurations, we should have achieved an accuracy of ~0.94, an increase of roughly 3 points compared to the original configuration of the [base tutorial](https://recognai.github.io/biome-text/v3.2.0/documentation/tutorials/1-Training_a_text_classifier.html).

In a real-life example, though, you probably should increase the number of epochs, since the validation loss in general seems to be decreasing further.

A next step could be to fix some of the tuned parameters to the preferred value, and tune other parameters further or limit their value space.

::: tip Tip

To obtain insights about the importance and tendencies of each hyperparameter for the model, we recommend using TensorBoard's *HPARAM* section and follow Richard Liaw's suggestions at the end of his [talk](https://www.youtube.com/watch?v=VX7HvEoMrsA). 
Another very useful tool is the parallel coordinates panel in [Weights & Biases](https://wandb.ai/site), see this [quick walkthrough](https://youtu.be/91HhNtmb0B4) 

:::

### Evaluating the best performing model

The *analysis* object also provides some convenient methods to obtain the best performing configuration, as well as the `logdir` where the results of the trial are saved.

In [ ]:
analysis.get_best_config("validation_accuracy", "max")

We can use the `best_logdir` to create a pipeline with the best performing model and start making predictions.

In [ ]:
best_logdir = analysis.get_best_logdir("validation_accuracy", "max")
best_model = os.path.join(best_logdir, "output", "model.tar.gz")
pl_trained = Pipeline.from_pretrained(best_model)

In [ ]:
pl_trained.predict(text="Autohaus Recognai")

::: tip Note

For an unbiased evaluation of the model you should use a test dataset that was not used during the HPO!

:::